In [1]:
import sys
import math
import datetime
from os import path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pydot
import tensorflow as tf
from tensorflow.python.keras.backend import dtype
from tensorflow.python.keras.layers.core import Lambda
from tensorflow import keras
from tensorflow.keras.utils import plot_model
from tensorflow.keras import losses
from tensorflow.keras.layers import *

class Dataloader:
    def __init__(
        self,
        input_width,
        label_width,
        shift=1,
        train_ratio=0.6,
        val_ratio=0.2,
        test_ratio=0.2,
        batch_size=256,
    ):
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift
        self.total_window_size = self.input_width + self.shift
        self.input_slice = slice(0, self.input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]
        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]
        self.batch_size = batch_size
        self.scale_coef = 1000.0
        self.bias = 0.0
        self.clip_value = 4.0

    def load_tsv(
        self,
        tsv_filename,
        input_column="open",
        train_ratio=0.6,
        val_ratio=0.2,
        test_ratio=0.2,
        verbose=1,
    ):
        df = pd.read_csv(
            tsv_filename,
            sep="\t",
            header=0,
            dtype={
                "open": np.float32,
                "close": np.float32,
                "tickvol": np.float32,
                "vol": np.float32,
            },
            names=["date","time","open","high","low","close","tickvol","vol","spread",],
        )
        df_size = df[input_column].size
        train_size = int(df_size * train_ratio)
        val_size = int(df_size * val_ratio)
        test_size = int(df_size * test_ratio)
        train_slice = slice(-train_size, None)
        val_slice = slice(-(train_size + val_size), -train_size)
        test_slice = slice(
            -(train_size + val_size + test_size), -(train_size + val_size)
        )
        self.train_df = df[input_column][train_slice]
        self.val_df = df[input_column][val_slice]
        self.test_df = df[input_column][test_slice]
        if verbose == 1:
            print(self.__sizes__())
            print(self.__repr__())
        return True

    def load_df(
        self,
        df: pd.core.frame.DataFrame,
        input_column="open",
        train_ratio=0.6,
        val_ratio=0.2,
        test_ratio=0.2,
        verbose=1,
    ):
        """
        Преобразует dataframe в обучающую выборку
        dataframe - Pandas Dataframe с колонками ["date", "time", "open", "high", "low", "close", "tickvol", "spread", "real_volume"]
        input_column - колонка с основными данными
        """
        df_size = df[input_column].size
        train_size = int(df_size * train_ratio)
        val_size = int(df_size * val_ratio)
        test_size = int(df_size * test_ratio)
        train_slice = slice(-train_size, None)
        val_slice = slice(-(train_size + val_size), -train_size)
        test_slice = slice(
            -(train_size + val_size + test_size), -(train_size + val_size)
        )
        self.train_df = df[input_column][train_slice]
        self.val_df = df[input_column][val_slice]
        self.test_df = df[input_column][test_slice]
        if verbose == 1:
            print(self.__sizes__())
            print(self.__repr__())
        return True

    def __sizes__(self):
        return "\n".join(
            [
                f"Размер train: {len(self.train_df)}",
                f"Размер validation: {len(self.val_df)}",
                f"Размер test: {len(self.test_df)}",
            ]
        )

    def __repr__(self):
        return "\n".join(
            [
                f"Размер окна: {self.total_window_size}",
                f"Размер входа: {self.input_width}",
                f"Размер выхода: {self.label_width}",
            ]
        )

    def make_dataset(self, data):
        ds = self.transform(data)
        ds = tf.keras.preprocessing.timeseries_dataset_from_array(
            data=ds,
            targets=None,
            sequence_length=self.total_window_size,
            sequence_stride=1,
            shuffle=True,
            batch_size=self.batch_size,
        )
        ds = ds.map(self.split_window)
        return ds

    def make_input(self, data):
        ds = self.transform(data)
        ds = tf.keras.preprocessing.timeseries_dataset_from_array(
            data=ds,
            targets=None,
            sequence_length=self.input_width,
            sequence_stride=1,
            shuffle=False,
        )
        return ds

    def make_output(self, output_data):
        d = self.inverse_transform(output_data)
        return d

    def split_window(self, databatch):
        inputs = databatch[:, self.input_slice]
        labels = databatch[:, self.labels_slice]
        inputs.set_shape([None, self.input_width])
        labels.set_shape([None, self.label_width])
        return inputs, labels

    def transform(self, data):
        # ds = tf.math.subtract(data, data[:, 0:1])
        # self.first_value = data[0:1]
        ds = np.diff(data) * self.scale_coef + self.bias
        std = np.std(ds)
        ds = np.clip(ds, -std * self.clip_value, std * self.clip_value)
        # ds = np.concatenate((ds[0:1], ds))
        # ds = np.diff(ds) * self.scale_coef + self.bias
        # ds = np.log(ds)
        return ds

    def inverse_transform(self, output_data):
        d = (output_data - self.bias) / self.scale_coef
        return d

    def moving_average(self, a, n=1):
        if n == 0:
            return a
        result = np.convolve(a, np.ones((n,)) / float(n), mode="valid")
        return result

    def shift_to_zero(self, data):
        data = tf.math.subtract(data, data[:, 0:1])  # сдвиг начального значения в ноль
        # data, data[:, self.input_width - 1 : self.input_width, :]
        return data

    @property
    def train(self):
        return self.make_dataset(self.train_df)

    @property
    def val(self):
        return self.make_dataset(self.val_df)

    @property
    def test(self):
        return self.make_dataset(self.test_df)

class ClippedMSE(losses.Loss):
    def __init__(
        self,
        value_min=-1.0,
        value_max=1.0,
        reduction=losses.Reduction.AUTO,
        name="clipped_mse",
    ) -> None:
        super().__init__(reduction=reduction, name=name)
        self.value_min = value_min
        self.value_max = value_max

    def call(self, y_true, y_pred):
        clipped_y_pred = tf.clip_by_value(y_pred, self.value_min, self.value_max)
        clipped_y_true = tf.clip_by_value(y_true, self.value_min, self.value_max)
        return losses.mean_squared_error(clipped_y_true, clipped_y_pred)


class ClippedMAE(losses.Loss):
    def __init__(self, value_min=-1.0, value_max=1.0, reduction=losses.Reduction.AUTO, name="clipped_mae") -> None:
        super().__init__(reduction=reduction, name=name)
        self.value_min = value_min
        self.value_max = value_max

    def call(self, y_true, y_pred):
        clipped_y_pred = tf.clip_by_value(y_pred, self.value_min, self.value_max)
        clipped_y_true = tf.clip_by_value(y_true, self.value_min, self.value_max)
        return losses.mean_absolute_error(clipped_y_true, clipped_y_pred)

def ConvAdaptiveKernelSize(x, activation, filters=8, kernel_size=2, dropout=0.5):
    k_size = kernel_size if x.shape[-2] >= kernel_size else x.shape[-2]
    l2 = keras.regularizers.l2(1e-10)
    x = Conv1D(filters, k_size, padding="valid")(x)
    x = LayerNormalization()(x)
    # x = BatchNormalization()(x)
    x = Lambda(activation)(x)
    # x = Dropout(rate=dropout)(x)
    return x


def red(
    input_width,
    out_width,
    columns=16,
    lr=1e-2,
    min_v=-2,
    max_v=2,
    training=True,
    name="red",
):
    if training:
        dropout = 1.0 / 256.0
    else:
        dropout = 0

    init = keras.initializers.RandomUniform(-1024, 1024)
    l2 = keras.regularizers.L2(l2=1e-10)
    dct_length = input_width
    def f_mean(z): return tf.math.reduce_mean(z, 1, keepdims=True)
    def f_logtanh(x): return tf.math.log(tf.exp(1.0) + tf.abs(x)) * tf.tanh(x)
    def f_dct(x): return tf.signal.dct(x, n=dct_length, norm='ortho')
    # def f_dct(x): return tf.signal.mdct(
    # x, frame_length = 8, norm = 'ortho', pad_end = True)
    n = int(math.log2(input_width))
    filters = 32
    inputs = Input(shape=(input_width,))
    # key = [Lambda(lambda z: z[:, -(2 ** (i+1)) :])(inputs) for i in range(n)]
    # m = [Lambda(f_mean, name=f"mean{i}")(key[i]) for i in range(n)]
    # m = Concatenate(name=f"concat_means")(m)
    # m = Dense(filters)(m)
    # m = Reshape((1, -1))(m)
    f = Lambda(f_dct, name=f"dct")(inputs)
    f = Reshape((-1, 1))(f)
    while f.shape[-2] > 1:
        f = ConvAdaptiveKernelSize(f, tf.nn.tanh, filters, 16, dropout)
    # f = Dropout(rate=dropout)(f)
    # x = Multiply()([m, f])
    # x = Flatten()(f)
    # x = Dense(64)(x)
    x = Reshape((-1, 1))(f)
    x = LSTM(64, return_sequences=True, dropout=dropout, name="lstm-1")(x)
    x = Flatten()(x)
    x = Dense(32, name=f"d-in-0")(x)
    rows_count = 8
    units = 32
    z = [Dense(units, name=f"d-in{c}-{0}")(x) for c in range(columns)]
    z = [Lambda(f_logtanh)(z[c]) for c in range(columns)]
    for c in range(columns):
        for r in range(rows_count - 1):
            z[c] = Dense(units, name=f"d{c}-{r}")(z[c])
            z[c] = BatchNormalization()(z[c])g
            z[c] = Lambda(f_logtanh)(z[c])
        z[c] = Dense(out_width)(z[c])
        # z[c] = Lambda(f_logtanh)(z[c])
    x = Concatenate()(z)
    x = Dense(out_width)(x)
    # x = Lambda(f_logtanh)(x)
    outputs = x
    model = keras.Model(inputs, outputs, name=name)
    MAE = keras.metrics.MeanAbsoluteError()
    CMSE = ClippedMSE(min_v, max_v)
    CMAE = ClippedMAE(min_v, max_v)
    model.compile(
        # loss=keras.losses.Huber(),
        # loss=keras.losses.MeanSquaredError(),
        loss=CMSE,
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        metrics=[MAE],
    )
    return model


2021-12-27 21:52:23.061786: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-27 21:52:23.061819: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
class Predictor(object):
    def __init__(
        self,
        datafile,
        model,
        input_width,
        label_width,
        shift=None,
        train_ratio=0.8,
        val_ratio=0.1,
        test_ratio=0.1,
        batch_size=256,
    ):
        if shift == None:
            shift = label_width
        self.dataloader = Dataloader(
            input_width=input_width,
            label_width=label_width,
            shift=shift,
            batch_size=batch_size,
        )
        if type(datafile) == str:
            self.dataloader.load_tsv(
                datafile,
                input_column="open",
                train_ratio=train_ratio,
                val_ratio=val_ratio,
                test_ratio=test_ratio,
            )
        elif type(datafile) == pd.core.frame.DataFrame:
            self.dataloader.load_df(
                datafile,
                input_column="open",
                train_ratio=train_ratio,
                val_ratio=val_ratio,
                test_ratio=test_ratio,
            )
        if type(model) == str:
            self.load_model(model)
        elif isinstance(model, tf.keras.Model):
            self.model = model
        else:
            print(
                "Ошибка загрузки модели модели. Параметр model должен быть либо строкой, либо моделью keras"
            )

    def __call__(self, data):
        # x = self.dataloader.make_input(data)
        return self.model(data[-self.dataloader.input_width - 1 :])

    def load_model(self, filename, lr=1e-5):
        # self.model = keras.models.load_model(self.name, custom_objects={"shifted_mse": shifted_mse})
        self.model = keras.models.load_model(filename, compile=False)
        self.model.compile(
            loss="mse",
            optimizer=keras.optimizers.Adam(learning_rate=lr),
            metrics=[keras.metrics.MeanAbsoluteError()],
        )
        return self.model

    def save_model(self):
        # self.model.save("models/" + self.model.name + ".h5")
        self.model.save("models/" + self.model.name)

    def plot(self):
        model_png_name = "models/" + self.model.name + ".png"
        plot_model(
            self.model,
            show_shapes=False,
            show_layer_names=False,
            rankdir="LR",
            to_file=model_png_name,
        )

    def fit(
        self,
        batch_size=256,
        epochs=2,
        use_tensorboard=True,
        use_early_stop=True,
        use_checkpoints=True,
        use_multiprocessing=True,
        verbose=1,
    ):
        callbacks = []
        start_fit_time = datetime.datetime.now()
        log_dir = "logs/fit/" + start_fit_time.strftime("%Y_%m_%d-%H_%M_%S")
        if use_checkpoints:
            ckpt = "ckpt/" + self.model.name + ".ckpt"
            backup = keras.callbacks.ModelCheckpoint(
                filepath=ckpt,
                monitor="loss",
                save_weights_only=True,
                save_best_only=True,
            )
            callbacks.append(backup)
            try:
                self.model.load_weights(ckpt)
                if verbose > 0:
                    print(
                        "Загружены веса последней контрольной точки " + self.model.name
                    )
            except Exception as e:
                pass

        if use_tensorboard:
            tb = keras.callbacks.TensorBoard(log_dir=log_dir, write_graph=True)
            callbacks.append(tb)
        if use_early_stop:
            es = keras.callbacks.EarlyStopping(
                monitor="val_loss",
                patience=2 ** 4,
                min_delta=1e-5,
                restore_best_weights=True,
            )
            callbacks.append(es)

        history = self.model.fit(
            self.dataloader.train,
            validation_data=self.dataloader.val,
            batch_size=batch_size,
            epochs=epochs,
            shuffle=True,
            use_multiprocessing=use_multiprocessing,
            verbose=verbose,
            callbacks=callbacks,
        )
        end_fit_time = datetime.datetime.now()
        delta_time = end_fit_time - start_fit_time
        if verbose > 0:
            print(f"\nВремя {start_fit_time}->{end_fit_time} : {delta_time}")
        return history

    def evaluate(self):
        return self.model.evaluate(self.dataloader.test, verbose=1)

    def predict(self, data, verbose=0):
        """Вычисление результата для набора data - массив размерности n"""
        x = self.dataloader.make_input(data)
        f = self.model.predict(x, use_multiprocessing=True, verbose=verbose)
        y = self.dataloader.make_output(f)
        # result = self.dataloader.make_output(y)
        return y

    def iterate(self, inputs, steps=1):
        results = []
        # input_width + 1 нужно для вычисления np.diff
        size = self.dataloader.input_width + 1
        for i in range(0, steps):
            inputs = inputs[-size:]
            output = float(self.predict(inputs, verbose=0)[-1][0])
            inputs = np.append(inputs, inputs[-1] + output)
            results.append(output)
        return results



In [ ]:
batch_size = 2 ** 10
dataset_segment = 1.0 / 8.0
input_width = 2 ** 8
label_width = 1
columns = 64

model = red(
    input_width,
    label_width,
    columns_count=columns,
    lr=1e-4,
    min_v=-2.0,
    max_v=2.0,
    training=True,
    name=f"eurusd-{columns}-{input_width}-{label_width}",
)

predictor = Predictor(
    datafile="datas/EURUSD_H1.csv",
    model=model,
    input_width=input_width,
    label_width=label_width,
    train_ratio=1.0 - 1.0 * dataset_segment,
    val_ratio=dataset_segment,
    test_ratio=0,
    batch_size=batch_size,
)

# predictor.plot()
# predictor.model.summary()

i = 0
while True:
    i += 1
    print(f"\nМодель {model.name} проход №{i}\n")
    history = predictor.fit(
        use_tensorboard=False,
        use_early_stop=False,
        batch_size=batch_size,
        epochs=2 ** 16,
    )
    predictor.save_model()
    # perfomance = predictor.evaluate()
    print("Модель обновлена")